# 🎭 ROOP Batch Processor - Google Colab

**Ejecuta las celdas en orden. La celda 1 puede pedir reinicio.**

In [ ]:
# ==========================================
# 📦 CELDA 1: Instalación completa
# ==========================================

# Clonar repo
!git clone https://github.com/CDavidDv/RoopConClaude.git 2>/dev/null || echo "Ya clonado"
%cd /content/RoopConClaude

# Instalar requirements
!pip install -r requirements-headless.txt

# Reinstalar versiones correctas
!pip uninstall -y onnxruntime onnxruntime-gpu insightface -q
!pip install insightface==0.7.3 onnxruntime-gpu==1.18.0 -q
!pip install tensorflow==2.12.0 -q

# Instalar libcufft10 para CUDA
!apt-get update -qq
!apt-get install -y -qq libcufft10

# Instalar numpy correcto
!pip install numpy==1.23.5 -q

print("\n" + "="*50)
print("✅ Instalación completada")
print("="*50)
print("\n⚠️  Si aparece error de numpy, reinicia el runtime")
print("   (Runtime > Restart runtime) y ejecuta CELDA 2")

In [ ]:
# ==========================================
# 🔍 CELDA 2: Verificar instalación
# ==========================================

%cd /content/RoopConClaude

import numpy as np
print(f"NumPy: {np.__version__}")

import onnxruntime as ort
print(f"ONNX Providers: {ort.get_available_providers()}")

import insightface
app = insightface.app.FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider'])
app.prepare(ctx_id=0)
print("✅ InsightFace con CUDA listo")

import torch
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

print("\n✅ Todo listo! Continúa con CELDA 3")

In [ ]:
# ==========================================
# 📤 CELDA 3: Subir archivos
# ==========================================

from google.colab import files
import os

%cd /content/RoopConClaude
!mkdir -p source inputVideos outputVideos

print("📸 Sube tu imagen de referencia:")
uploaded = files.upload()
for f in uploaded:
    os.rename(f, f"source/{f}")
    print(f"✅ source/{f}")

print("\n🎬 Sube tus videos:")
uploaded = files.upload()
for f in uploaded:
    os.rename(f, f"inputVideos/{f}")
    print(f"✅ inputVideos/{f}")

print("\n✅ Archivos listos!")

In [ ]:
# ==========================================
# 🚀 CELDA 4: Procesar videos
# ==========================================

%cd /content/RoopConClaude

# Procesar TODOS los videos
!python runbatch.py

# O procesar un RANGO (descomenta):
# !python runbatch.py 1-40

In [ ]:
# ==========================================
# 📥 CELDA 5: Descargar resultados
# ==========================================

from google.colab import files
from pathlib import Path

%cd /content/RoopConClaude

for video in sorted(Path('outputVideos').glob('*.mp4')):
    print(f"⬇️ {video.name}")
    files.download(str(video))

print("\n✅ Descarga completada!")

In [ ]:
# ==========================================
# 📤 EXTRA: Subir más videos
# ==========================================

from google.colab import files
import os

%cd /content/RoopConClaude

print("🎬 Sube más videos:")
uploaded = files.upload()
for f in uploaded:
    os.rename(f, f"inputVideos/{f}")
    print(f"✅ inputVideos/{f}")